In [ ]:
!pip install -q rpy2
!pip install scanpy
from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
Mounted at /content/drive


In [ ]:
import scanpy as sc
import pandas as pd

# # 读取 .h5ad 文件
# adata = sc.read_h5ad("/content/drive/MyDrive/PBMC_simulated_cnas_041025.h5ad")

# # 转换为 gene × cell 的表达矩阵，CopyKAT 要求这样格式
# expr_matrix = pd.DataFrame(adata.X.T.toarray(), index=adata.var_names, columns=adata.obs_names)


In [ ]:

adata = sc.read_h5ad("/content/drive/MyDrive/PBMC_simulated_cnas_041025.h5ad")

# 转换为 gene × cell 的表达矩阵，CopyKAT 要求这样格式
expr_matrix = pd.DataFrame(adata.X.T.toarray(), index=adata.var_names, columns=adata.obs_names)


In [ ]:
%load_ext rpy2.ipython


In [ ]:
%%R
# 安装 BiocManager
if (!requireNamespace("BiocManager", quietly = TRUE)) {
  install.packages("BiocManager")
}

# 安装 CopyKAT（从 GitHub）
BiocManager::install("navinlabcode/copykat", ask = FALSE, update = FALSE)

# 加载库
library(copykat)


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpvrMzlz/remotes21262a0d8ab/navinlabcode-copykat-d7d6569/DESCRIPTION’ ... OK
* preparing ‘copykat’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
  NB: this package now depends on R (>= 3.5.0)
  serialize/load version 3 cannot be read in older versions of R.
  File(s) containing such objects:
    ‘copykat/data/sysdata.rda’
* building ‘copykat_1.1.0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.25.tar.gz'
Content type 'application/x-gzip' length 593414 bytes (579 KB)
downloaded 579 KB


The downloaded source packages are in
	‘/tmp/RtmpvrMzlz/downloaded_packages’
'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.21 (BiocManager 1.30.25), R 4.5.0 (2025-04-11)
Installing package(s) 'BiocVersion'
trying URL 'https://bioconductor.org/packages/3.21/bioc/src/contrib/BiocVersion_3.21.1.tar.gz'
Content type 'application/x-gzip' length 991 bytes
downloaded 991 bytes


The downloaded source packages are in
	‘/tmp/RtmpvrMzlz/downloaded_packages’
Installing github package(s) 'navinlabcode/copykat'
Installing 20 packages: MatrixModels, SparseM, crosstalk, lazyeval, bitops, quantreg,

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# 传入 R 环境
robjects.globalenv['expr_matrix'] = pandas2ri.py2rpy(expr_matrix)


In [ ]:
%%R
results <- copykat(
  rawmat = expr_matrix,
  id.type = "SYMBOL",
  ngene.chr = 5,
  win.size = 250,
  norm.cell.names = NULL,
  output.seg = "no"
)

# 查看分类结果
head(results$prediction)


[1] "running copykat v1.1.0"
[1] "step1: read and filter data ..."
[1] "20104 genes, 10309 cells in raw data"
[1] "8612 genes past LOW.DR filtering"
[1] "step 2: annotations gene coordinates ..."
[1] "start annotation ..."
[1] "step 3: smoothing data with dlm ..."
[1] "step 4: measuring baselines ..."
number of iterations= 127 
number of iterations= 352 
number of iterations= 167 
number of iterations= 105 
number of iterations= 161 
number of iterations= 256 
[1] "low confidence in classification"
[1] "cell: 1"
WARNING! NOT CONVERGENT! 
number of iterations= 500 
[1] "cell: 2"
WARNING! NOT CONVERGENT! 
number of iterations= 500 
[1] "cell: 3"
WARNING! NOT CONVERGENT! 
number of iterations= 500 
[1] "cell: 4"
number of iterations= 198 
[1] "cell: 5"
WARNING! NOT CONVERGENT! 
number of iterations= 500 
[1] "cell: 6"
WARNING! NOT CONVERGENT! 
number of iterations= 500 
[1] "cell: 7"
WARNING! NOT CONVERGENT! 
number of iterations= 500 
[1] "cell: 8"
WARNING! NOT CONVERGENT! 
number of ite

In [ ]:
copykat_pred = pandas2ri.rpy2py(robjects.r('results$prediction'))
copykat_pred = pd.DataFrame(copykat_pred)
copykat_pred.columns = ['copykat_prediction']
copykat_pred.index.name = 'cell'

# 合并回 AnnData 对象中
adata.obs['copykat'] = copykat_pred.loc[adata.obs_names, 'copykat_prediction'].values

# 查看统计
adata.obs['copykat'].value_counts()
